In [39]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_AWZDTJC7ELLLWKQJM7XSDOUT-QMTVJN4LDJALI4EQ")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [40]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5


In [41]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    
    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

In [42]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.strip().upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 0.95s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [43]:

def wordsOfLength(lengths):
    return [word for word in filteredWords if len(word) in lengths]

len(list(wordsOfLength(START_WORD_LENGTHS)))


35094

In [44]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False
    
    res = True
    
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break
    
    return res
    
        
assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [45]:
def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False
        
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet
        
    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
    
    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res
    
        
maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))


isAnagramEx("straccio", "crosta", 2)
True


In [46]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



Build took 3.38369 s
Read 401123 words into 674739 nodes
Search took 0.934999 s


In [47]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: CIRCA
Search took 0.32102 s
END WORD: ('CUPA', 3)


In [48]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        if isAnagramEx(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))
    
    return results


In [49]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])
    
print(anagrams)
#print(anagramsDict[0][:10])

START WORD: CIRCA
[('ACCASARSI', 0), ('ACCORASSI', 0), ('ACCORSASI', 0), ('ACCURASSI', 0), ('ACCUSARSI', 0), ('BRACCASSI', 0), ('CARCASSI', 1), ('CARICASSE', 0), ('CARICASSI', 0), ('CASCARSI', 1), ('CASSARCI', 1), ('CIRCASSA', 1), ('CLASSARCI', 0), ('CRACCASSI', 0), ('CRACKARSI', 0), ('CRACKASSE', 0), ('CRACKASTI', 0), ('RACCASSI', 1), ('RACCESASI', 0), ('RICASSACI', 0), ('SACCARASI', 0), ('SACCAROSI', 0), ('SARACCOSI', 0), ('SCALCARSI', 0), ('SCANSARCI', 0), ('SCARNISCA', 0), ('SCASARCI', 1), ('SCASSARCI', 0), ('SFRASCACI', 0), ('SMACCARSI', 0), ('SPACCARSI', 0), ('STACCARSI', 0), ('STRASCICA', 0), ('SVACCARSI', 0)]


In [50]:
print(anagramsDict[1])

['CARCASSI', 'CASCARSI', 'CASSARCI', 'CIRCASSA', 'RACCASSI', 'SCASARCI']


In [51]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTHS)
    
    restart = True
    
    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)
    
        steps = [startWord]
    
        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break
                    
            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)
    
                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True
    
            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep
    
        print("restart", restart)
        print()
    
    endWord = steps[-1]
    print("END WORD:", endWord)  
    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

START WORD: CUBIA
Finding anagrams... (0.27)
('CUBATI', 1) 46 anagram
Finding levenshtein... (0.01)
('BUBATI', 1) 20 levenshtein
Finding levenshtein... (0.01)
('BUBANTI', 1) 18 levenshtein
Finding anagrams... (2.00)
anagram 5
restart True

START WORD: RIRIDE
Finding levenshtein... (0.04)
('RIGIDE', 1) 12 levenshtein
Finding levenshtein... (0.03)
levenshtein 9
restart True

START WORD: COFTO
Finding anagrams... (0.47)
anagram 7
restart True

START WORD: SUFITA
Finding anagrams... (0.43)
('AFFUSTI', 1) 44 anagram
Finding anagrams... (0.73)
('EFFUSATI', 1) 16 anagram
Finding levenshtein... (0.00)
levenshtein 9
restart True

START WORD: COLALO
Finding levenshtein... (0.01)
('COLALE', 1) 22 levenshtein
Finding anagrams... (0.21)
('CLONALE', 1) 46 anagram
Finding anagrams... (0.44)
('LANCIOLE', 1) 30 anagram
Finding levenshtein... (0.01)
levenshtein 5
restart True

START WORD: ODILO
Finding levenshtein... (0.00)
('RODILO', 1) 13 levenshtein
Finding levenshtein... (0.01)
('GODILO', 1) 15 leve

In [52]:
def isInDictionary(word):
    results = search(word, 0)
    return len(results) > 0

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    for word in wordChain + [endWord]:
        inDictionary = isInDictionary(word)
        results = [t[0] for t in search(prevWord, 1)]
        anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        isWordOk = inDictionary and (word in results or word in anagrams) 
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)
        
        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses
            
print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: BOERI
END WORD  : MATTO


In [53]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [54]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: ELGEYE
levenshtein 4
restart True

START WORD: ALARMI
levenshtein 9
restart True

START WORD: NERINA
levenshtein 5
restart True

START WORD: FORI
restart False

END WORD: GIRSI

Generate 1.20s
START WORD: FORI
END WORD  : GIRSI


In [55]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))


FORI
❌ PANTERA
❌ PENTOLA
❌ PANTOFOLA
❌ OSTERIA
❌ RISTORAI
❌ PIANTALA
❌ GIRSI
GIRSI

👎 LOST

Validate 4.20s


In [56]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

COMPUTED WORD CHAIN ['FORI', 'ZIRO', 'ZIRI', 'DIRI', 'SIRI', 'SCIRI', 'GIRSI']


In [57]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


[('MUOVI', 0)]

In [58]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

['0x2716', '0xfe0f']
['0x2714', '0xfe0f']
['0x274c']


'0x1f44e'

In [59]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



In [ ]:
anvil.server.wait_forever()